# 📘 Архитектуры Агентов 7: Blackboard Systems (Системы с классной доской)

Добро пожаловать в седьмой блокнот нашей серии об агентных архитектурах. Сегодня мы исследуем **Blackboard System**, мощный и очень гибкий паттерн для координации множества узкоспециализированных агентов. Эта архитектура смоделирована по идее группы экспертов-людей, сотрудничающих у физической классной доски для решения сложной проблемы.

Вместо жесткой, предопределенной последовательности передачи задач агентами, Blackboard system предлагает центральное, общее хранилище данных ('blackboard'), где агенты могут читать текущее состояние проблемы и записывать свои вклады. Динамический **Controller** наблюдает за доской и решает, какого агента-специалиста активировать следующим, основываясь на том, что необходимо для продвижения решения вперед. Это позволяет создать оппортунистический и эмерджентный рабочий процесс.

Чтобы подчеркнуть его уникальные преимущества, мы сравним его с **последовательной мульти-агентной системой**, которую мы построили ранее. Мы представим обеим системам сложный финансовый запрос, где оптимальный путь — это не простая последовательность A → B → C. Мы продемонстрируем, как жесткий последовательный агент следует неоптимальному пути, в то время как динамический Controller системы Blackboard активирует агентов в более логичном порядке, управляемом данными, что приводит к более эффективному и связному анализу.

### Определение
**Blackboard System** — это мульти-агентная архитектура, в которой несколько специализированных агентов сотрудничают, читая и записывая в общее, центральное хранилище данных, называемое 'blackboard'. Контроллер или планировщик динамически определяет, какой агент должен действовать следующим, основываясь на изменяющемся состоянии решения на доске.

### Высокоуровневый рабочий процесс

1.  **Общая Память (The Blackboard):** Центральная структура данных хранит текущее состояние проблемы, включая запрос пользователя, промежуточные находки и частичные решения.
2.  **Агенты-Специалисты (Specialist Agents):** Пул независимых агентов, каждый со своей специфической экспертизой, непрерывно мониторит доску.
3.  **Контроллер (Controller):** Центральный агент 'controller' также мониторит доску. Его работа — анализировать текущее состояние и решать, какой агент-специалист лучше всего подходит для внесения следующего вклада.
4.  **Оппортунистическая Активация:** Контроллер активирует выбранного агента. Агент читает релевантные данные с доски, выполняет свою задачу и записывает свои находки обратно на доску.
5.  **Итерация:** Процесс повторяется, при этом Контроллер активирует разных агентов в динамической последовательности, пока не определит, что решение на доске завершено.

### Когда использовать / Применения
*   **Сложные, Слабоструктурированные проблемы:** Идеально подходит для проблем, где путь решения не известен заранее и требует эмерджентной, оппортунистической стратегии (например, сложная диагностика, научные открытия).
*   **Мультимодальные системы:** Отличный способ координировать агентов, работающих с разными типами данных (текст, изображения, код), так как все они могут публиковать свои находки на общей доске.
*   **Динамическое осмысление (Sense-Making):** Ситуации, требующие синтеза информации из множества разрозненных, асинхронных источников.

### Сильные и слабые стороны
*   **Сильные стороны:**
    *   **Гибкость и Адаптивность:** Рабочий процесс не закодирован жестко; он возникает на основе проблемы, делая систему высокоадаптивной.
    *   **Модульность:** Очень легко добавлять или удалять агентов-специалистов без перепроектирования всей системы.
*   **Слабые стороны:**
    *   **Сложность Контроллера:** Интеллект всей системы сильно зависит от искушенности Контроллера. Наивный Контроллер может привести к неэффективному или зацикленному поведению.
    *   **Сложности отладки:** Нелинейная, эмерджентная природа рабочего процесса иногда может делать его более трудным для отслеживания и отладки, чем простой последовательный процесс.

## Фаза 0: Основа и Настройка

Мы начнем с нашего стандартного процесса настройки: установки библиотек и конфигурации API ключей для Nebius, LangSmith и Tavily.

### Шаг 0.1: Установка основных библиотек

**Что мы будем делать:**
Мы установим наш стандартный набор библиотек для этой серии проектов.

In [ ]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv langchain-tavily

### Шаг 0.2: Импорт библиотек и настройка ключей

**Что мы будем делать:**
Мы импортируем необходимые модули и загрузим наши API ключи из файла `.env`.

**Требуемое действие:** Создайте файл `.env` в этой директории с вашими ключами:
```
NEBIUS_API_KEY="your_nebius_api_key_here"
LANGCHAIN_API_KEY="your_langsmith_api_key_here"
TAVILY_API_KEY="your_tavily_api_key_here"
```

In [ ]:
import os
import json
from typing import List, Annotated, TypedDict, Optional, Dict
from dotenv import load_dotenv

# Компоненты LangChain
from langchain_nebius import ChatNebius
from langchain_community.tools.tavily_search import TavilySearchResults as TavilySearch
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

# Компоненты LangGraph
from langgraph.graph import StateGraph, END

# Для красивого вывода
from rich.console import Console
from rich.markdown import Markdown

from notebook_utils import setup_environment

# --- Настройка API ключей и трассировки ---
console = setup_environment(
    project_name="Agentic Architecture - Blackboard (Nebius)",
    required_keys=["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "TAVILY_API_KEY"]
)

Environment variables loaded and tracing is set up.


## Фаза 1: Базовый уровень - Последовательная мульти-агентная система (Исправленная)

Чтобы понять гибкость Blackboard, нам сначала нужна корректно функционирующая последовательная система. Оригинальная версия терпела неудачу, потому что специалисты не использовали вывод предыдущих шагов. Мы исправим это, гарантируя, что каждый агент получает необходимый контекст из состояния.

### Шаг 1.1: Создание (Исправленной) Последовательной Команды

**Что мы будем делать:**
Мы определим агентов-специалистов, которые явно используют результаты своих предшественников, а затем свяжем их в фиксированную, линейную последовательность.

In [ ]:
console = Console()
# Использование более способной модели для лучшей обработки сложных инструкций
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0)
search_tool = TavilySearch(max_results=2)

# Состояние для последовательного агента
class SequentialState(TypedDict):
    user_request: str
    news_report: Optional[str]
    technical_report: Optional[str]
    financial_report: Optional[str]
    final_report: Optional[str]

# --- ИСПРАВЛЕННЫЕ УЗЛЫ СПЕЦИАЛИСТОВ ДЛЯ ПОСЛЕДОВАТЕЛЬНОГО АГЕНТА ---
# Ключевое изменение: каждый агент теперь получает контекст из предыдущих шагов, а не только оригинальный запрос.

def news_analyst_node_seq(state: SequentialState):
    console.print("--- (Sequential) ВЫЗОВ NEWS ANALYST ---")
    prompt = f"Your task is to act as an expert News Analyst. Find the latest major news about the topic in the user's request and provide a concise summary.\n\nUser Request: {state['user_request']}"
    agent = llm.bind_tools([search_tool])
    result = agent.invoke(prompt)
    return {"news_report": result.content}

def technical_analyst_node_seq(state: SequentialState):
    console.print("--- (Sequential) ВЫЗОВ TECHNICAL ANALYST ---")
    # Этот агент теперь использует отчет по новостям как контекст.
    prompt = f"Your task is to act as an expert Technical Analyst. Based on the following news report, conduct a technical analysis of the company's stock.\n\nNews Report:\n{state['news_report']}"
    agent = llm.bind_tools([search_tool])
    result = agent.invoke(prompt)
    return {"technical_report": result.content}

def financial_analyst_node_seq(state: SequentialState):
    console.print("--- (Sequential) ВЫЗОВ FINANCIAL ANALYST ---")
    # Этот агент также использует отчет по новостям как контекст.
    prompt = f"Your task is to act as an expert Financial Analyst. Based on the following news report, analyze the company's recent financial performance.\n\nNews Report:\n{state['news_report']}"
    agent = llm.bind_tools([search_tool])
    result = agent.invoke(prompt)
    return {"financial_report": result.content}


def report_writer_node_seq(state: SequentialState):
    console.print("--- (Sequential) ВЫЗОВ REPORT WRITER ---")
    prompt = f"""You are an expert report writer. Your task is to synthesize the information from the News, Technical, and Financial analysts into a single, cohesive report that directly answers the user's original request.

User Request: {state['user_request']}

Here are the reports to combine:
---
News Report: {state['news_report']}
---
Technical Report: {state['technical_report']}
---
Financial Report: {state['financial_report']}
"""
    report = llm.invoke(prompt).content
    return {"final_report": report}

# Построение последовательного графа
seq_graph_builder = StateGraph(SequentialState)
seq_graph_builder.add_node("news", news_analyst_node_seq)
seq_graph_builder.add_node("tech", technical_analyst_node_seq)
seq_graph_builder.add_node("finance", financial_analyst_node_seq)
seq_graph_builder.add_node("writer", report_writer_node_seq)

# Жесткая, закодированная последовательность
seq_graph_builder.set_entry_point("news")
seq_graph_builder.add_edge("news", "tech")
seq_graph_builder.add_edge("tech", "finance")
seq_graph_builder.add_edge("finance", "writer")
seq_graph_builder.add_edge("writer", END)

sequential_app = seq_graph_builder.compile()
print("Исправленная последовательная мульти-агентная система успешно скомпилирована.")

Corrected sequential multi-agent system compiled successfully.


### Шаг 1.2: Тестирование Последовательного Агента на Динамической проблеме

Теперь, когда последовательный агент корректно передает контекст, давайте понаблюдаем за его поведением. Он создаст более связный отчет, но его *процесс* все равно будет неэффективным и не сможет следовать условной логике.

In [ ]:
dynamic_query = "Find the latest major news about Nvidia. Based on the sentiment of that news, conduct either a technical analysis (if the news is neutral or positive) or a financial analysis of their recent performance (if the news is negative)."

console.print(f"[bold yellow]Тестирование ИСПРАВЛЕННОГО ПОСЛЕДОВАТЕЛЬНОГО агента на динамическом запросе:[/bold yellow]\n'{dynamic_query}'\n")

# Запуск графа
final_seq_output = sequential_app.invoke({"user_request": dynamic_query})

console.print("\n--- [bold red]Финальный отчет от Последовательного Агента[/bold red] ---")
console.print(Markdown(final_seq_output['final_report']))

Testing CORRECTED SEQUENTIAL agent on a dynamic query:
'Find the latest major news about Nvidia. Based on the sentiment of that news, conduct either a technical analysis (if the news is neutral or positive) or a financial analysis of their recent performance (if the news is negative).'


--- (Sequential) CALLING NEWS ANALYST ---
--- (Sequential) CALLING TECHNICAL ANALYST ---
--- (Sequential) CALLING FINANCIAL ANALYST ---
--- (Sequential) CALLING REPORT WRITER ---



--- Final Report from Sequential Agent ---


### Comprehensive Analysis of Nvidia Following Latest News

**User Request:** Find the latest major news about Nvidia. Based on the sentiment of that news, conduct either a technical analysis (if the news is neutral or positive) or a financial analysis of their recent performance (if the news is negative).

**1. News Summary:**
Recent major news concerning Nvidia revolves around the announcement of their next-generation AI chip architecture, codenamed "Rubin." This news has generated significant positive sentiment in the market, with analysts highlighting the potential for continued dominance in the AI hardware space. The announcement points to a roadmap extending beyond the current "Blackwell" chips, suggesting sustained innovation and growth. This news is overwhelmingly positive.

**2. Technical Analysis:**
Following the positive news, Nvidia's stock (NVDA) has shown strong bullish indicators. The stock price has broken through previous resistance levels and is trading well above its 50-day and 200-day moving averages. Trading volume has been high on up-days, confirming the bullish trend. Key indicators like the Relative Strength Index (RSI) are in overbought territory, suggesting a possible short-term pullback, but the overall momentum remains strongly positive.

**3. Financial Analysis:**
Nvidia's recent financial performance has been exceptionally strong, driven by surging demand for its data center GPUs. The company's latest quarterly earnings report showed revenue and net income figures that significantly beat analyst expectations. Gross margins have expanded, and the company has provided a very optimistic forecast for the upcoming quarter, further bolstering investor confidence.

**Conclusion:**
The combination of groundbreaking product announcements, positive market sentiment, strong technical stock performance, and record-breaking financial results paints a very bullish picture for Nvidia. While the user's request specified an 'either/or' analysis based on sentiment, this report includes all analyses for completeness, confirming the company's robust position from multiple perspectives.

**Обсуждение (Исправленного) Вывода:**
Агент теперь создает полный, логичный отчет. Однако трассировка выполнения `News → Technical → Financial` раскрывает его фундаментальный изъян. Он выполнил **оба** анализа, и технический, и финансовый, полностью игнорируя условный запрос пользователя ("либо... либо..."). Это неэффективно и демонстрирует негибкость, которую мы стремимся решить с помощью архитектуры Blackboard.

## Фаза 2: Продвинутый подход - Система Blackboard (Исправленная)

Теперь мы построим систему Blackboard. Ключом к исправлению зацикливания оригинала является создание гораздо более интеллектуального промпта для **Controller**, делая его осведомленным о своей роли планировщика с состоянием.

### Шаг 2.1: Определение Blackboard и (Исправленного) Контроллера

**Что мы будем делать:**
1.  **Состояние Blackboard:** Определим `BlackboardState` для общей памяти.
2.  **Агенты-Специалисты:** Определим узлы специалистов. Они будут похожи на наших предыдущих агентов.
3.  **Контроллер (Исправленный):** Создадим надежный `controller_node` с промптом, который явно рассуждает о завершенных шагах и оставшихся целях. Это самое критическое изменение.

In [ ]:
# Blackboard State хранит всю информацию
class BlackboardState(TypedDict):
    user_request: str
    # Центральная доска, где агенты публикуют свои находки в виде строк
    blackboard: List[str]
    # Список доступных агентов, из которых контроллер может выбирать
    available_agents: List[str]
    # Следующее решение контроллера
    next_agent: Optional[str]

# Pydantic модель для решения Контроллера
# ИСПРАВЛЕНИЕ: Добавлен список доступных агентов в описание поля, чтобы направить выбор LLM.
class ControllerDecision(BaseModel):
    next_agent: str = Field(description="The name of the next agent to call. Must be one of ['News Analyst', 'Technical Analyst', 'Financial Analyst', 'Report Writer'] or 'FINISH'.")
    reasoning: str = Field(description="A brief reason for choosing the next agent.")

# Переиспользуемая фабрика для создания агентов-специалистов для доски
def create_blackboard_specialist(persona: str, agent_name: str):
    system_prompt = f"""You are an expert specialist agent: a {persona}.
Your task is to contribute to a larger goal by performing your specific function.
Read the initial User Request and the current Blackboard for context.
Use your tools to find the required information.
Finally, post your concise markdown report back to the blackboard. Your report should be signed with your name '{agent_name}'.
"""
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "User Request: {user_request}\n\nBlackboard (previous reports):\n{blackboard_str}")
    ])
    agent = prompt_template | llm.bind_tools([search_tool])

    def specialist_node(state: BlackboardState):
        console.print(f"--- (Blackboard) AGENT '{agent_name}' работает... ---")
        blackboard_str = "\n---\n".join(state["blackboard"])
        result = agent.invoke({"user_request": state["user_request"], "blackboard_str": blackboard_str})
        report = f"**Report from {agent_name}:**\n{result.content}"
        # Добавляем новый отчет в список записей на доске
        return {"blackboard": state["blackboard"] + [report]}
    return specialist_node

# Создание узлов агентов-специалистов
news_analyst_bb = create_blackboard_specialist("News Analyst", "News Analyst")
technical_analyst_bb = create_blackboard_specialist("Technical Analyst", "Technical Analyst")
financial_analyst_bb = create_blackboard_specialist("Financial Analyst", "Financial Analyst")
report_writer_bb = create_blackboard_specialist("Report Writer who synthesizes a final answer from the blackboard", "Report Writer")

# --- ИСПРАВЛЕННЫЙ, ИНТЕЛЛЕКТУАЛЬНЫЙ УЗЕЛ КОНТРОЛЛЕРА ---
# Это самое важное исправление. Промпт теперь гораздо более сложный.
def controller_node(state: BlackboardState):
    console.print("--- CONTROLLER: Анализ доски... ---")

    # Использование structured output LLM для принятия решения
    controller_llm = llm.with_structured_output(ControllerDecision)

    blackboard_content = "\n\n".join(state['blackboard'])
    agent_list = state['available_agents']

    # Новый промпт учитывает состояние и ориентирован на цель.
    prompt = f"""You are the central controller of a multi-agent system. Your job is to analyze the shared blackboard and the original user request to decide which specialist agent should run next.

**Original User Request:**
{state['user_request']}

**Current Blackboard Content:**
---
{blackboard_content if blackboard_content else "The blackboard is currently empty."}
---

**Available Specialist Agents:**
{', '.join(agent_list)}

**Your Task:**
1.  Read the user request and the current blackboard content carefully.
2.  Determine what the *next logical step* is to move closer to a complete answer.
3.  Choose the single best agent to perform that step from the list of available agents.
4.  If the user's request has been fully addressed and a final report has been written, choose 'FINISH'. Do not finish until a "Report Writer" has provided a final, synthesized answer.

Provide your decision in the required format.
"""
    decision_result = controller_llm.invoke(prompt)
    console.print(f"--- CONTROLLER: Решение вызвать '{decision_result.next_agent}'. Причина: {decision_result.reasoning} ---")

    # Словарь, возвращаемый здесь, обновляет ключ 'next_agent' в состоянии графа
    return {"next_agent": decision_result.next_agent}

print("Компоненты Blackboard и исправленный узел Controller определены.")

Blackboard components and corrected Controller node defined.


### Шаг 2.2: Построение Графа Blackboard

Теперь мы связываем компоненты в динамический граф. Контроллер действует как центральный маршрутизатор. После работы любого специалиста управление всегда возвращается к Контроллеру для принятия решения о следующем шаге.

In [ ]:
bb_graph_builder = StateGraph(BlackboardState)

# Добавление всех узлов в граф
bb_graph_builder.add_node("Controller", controller_node)
bb_graph_builder.add_node("News Analyst", news_analyst_bb)
bb_graph_builder.add_node("Technical Analyst", technical_analyst_bb)
bb_graph_builder.add_node("Financial Analyst", financial_analyst_bb)
bb_graph_builder.add_node("Report Writer", report_writer_bb)

bb_graph_builder.set_entry_point("Controller")

# Эта функция определяет динамическую логику маршрутизации на основе решения Контроллера
def route_to_agent(state: BlackboardState):
    return state["next_agent"]

# Условные ребра маршрутизируют от Контроллера к выбранному специалисту или к концу
bb_graph_builder.add_conditional_edges(
    "Controller",
    route_to_agent,
    {
        "News Analyst": "News Analyst",
        "Technical Analyst": "Technical Analyst",
        "Financial Analyst": "Financial Analyst",
        "Report Writer": "Report Writer",
        "FINISH": END
    }
)

# После работы любого специалиста управление всегда возвращается к Контроллеру для следующего решения
bb_graph_builder.add_edge("News Analyst", "Controller")
bb_graph_builder.add_edge("Technical Analyst", "Controller")
bb_graph_builder.add_edge("Financial Analyst", "Controller")
bb_graph_builder.add_edge("Report Writer", "Controller")

blackboard_app = bb_graph_builder.compile()
print("Система Blackboard успешно скомпилирована.")

Blackboard system compiled successfully.


## Фаза 3: Прямое сравнение

Давайте запустим нашу новую систему Blackboard на той же самой динамической задаче и понаблюдаем за её интеллектуальным рабочим процессом.

In [ ]:
console.print(f"[bold green]Тестирование системы BLACKBOARD на том же динамическом запросе:[/bold green]\n'{dynamic_query}'\n")

agent_list = ["News Analyst", "Technical Analyst", "Financial Analyst", "Report Writer"]
initial_bb_input = {"user_request": dynamic_query, "blackboard": [], "available_agents": agent_list}

# Мы используем stream, чтобы наблюдать за пошаговым процессом
final_bb_output = None
for chunk in blackboard_app.stream(initial_bb_input, {"recursion_limit": 10}):
    final_bb_output = chunk
    console.print("\n--- [bold purple]Текущее состояние доски[/bold purple] ---")
    # Красивый вывод каждого отчета на доске
    for i, report in enumerate(final_bb_output.get('blackboard', [])):
        console.print(f"--- Report {i+1} ---")
        console.print(Markdown(report))
    console.print("\n")

console.print("\n--- [bold green]Финальный отчет от Системы Blackboard[/bold green] ---")
# Финальный отчет - это последний элемент, опубликованный на доске писателем
final_report_content = final_bb_output['blackboard'][-1]
console.print(Markdown(final_report_content))

Testing BLACKBOARD system on the same dynamic query:
'Find the latest major news about Nvidia. Based on the sentiment of that news, conduct either a technical analysis (if the news is neutral or positive) or a financial analysis of their recent performance (if the news is negative).'


--- CONTROLLER: Analyzing blackboard... ---
--- CONTROLLER: Decision is to call 'News Analyst'. Reason: The blackboard is empty, so the first step is to gather the latest news about Nvidia as requested by the user. ---



--- Current Blackboard State ---



--- (Blackboard) AGENT 'News Analyst' is working... ---
--- CONTROLLER: Analyzing blackboard... ---
--- CONTROLLER: Decision is to call 'Technical Analyst'. Reason: The news report on the blackboard is positive, mentioning a new chip architecture and market dominance. According to the user's request, a technical analysis is the correct next step for positive news. ---



--- Current Blackboard State ---


...

**Report from Technical Analyst:**
Based on the positive news regarding Nvidia's accelerated roadmap, a technical analysis of NVDA stock shows strong bullish momentum. The stock is currently in a clear uptrend, trading significantly above its 50-day and 200-day simple moving averages (SMAs). The announcement created a bullish gap on the price chart, which has held, indicating strong buying pressure. The Relative Strength Index (RSI) is high, suggesting the stock is in overbought territory, but this is common for a stock in a strong uptrend. Key support is now established at the top of the recent price gap.

--- Report 3 ---


**Report from Report Writer:**
### Final Analysis of Nvidia Based on Recent News

**Initial Request:** Find the latest major news about Nvidia. Based on the sentiment of that news, conduct either a technical analysis (if the news is neutral or positive) or a financial analysis of their recent performance (if the news is negative).

**Process Summary:**
1.  **News Analysis:** The latest major news for Nvidia is the announcement of its next-generation "Rubin" AI chip platform, which has generated highly positive market sentiment.
2.  **Conditional Action:** As per the user's request, the positive news sentiment triggered a technical analysis.
3.  **Technical Analysis:** The technical outlook for NVDA stock is strongly bullish. The stock is in a clear uptrend, trading above key moving averages, with significant buying pressure confirmed by a recent price gap.

**Conclusion:** Following the user's instructions, an analysis of recent positive news about Nvidia's accelerated innovation led to a technical review, which confirms a strong bullish momentum for the company's stock.


--- Final Report from Blackboard System ---


**Report from Report Writer:**
### Final Analysis of Nvidia Based on Recent News

**Initial Request:** Find the latest major news about Nvidia. Based on the sentiment of that news, conduct either a technical analysis (if the news is neutral or positive) or a financial analysis of their recent performance (if the news is negative).

**Process Summary:**
1.  **News Analysis:** The latest major news for Nvidia is the announcement of its next-generation "Rubin" AI chip platform, which has generated highly positive market sentiment.
2.  **Conditional Action:** As per the user's request, the positive news sentiment triggered a technical analysis.
3.  **Technical Analysis:** The technical outlook for NVDA stock is strongly bullish. The stock is in a clear uptrend, trading above key moving averages, with significant buying pressure confirmed by a recent price gap.

**Conclusion:** Following the user's instructions, an analysis of recent positive news about Nvidia's accelerated innovation led to a technical review, which confirms a strong bullish momentum for the company's stock.

**Обсуждение (Исправленного) Вывода:**
Успех! Ошибка `GraphRecursionError` исчезла. Трассировка выполнения раскрывает гораздо более интеллектуальный процесс:

1.  **Старт Контроллера:** Контроллер стартует и, видя пустую доску, верно решает вызвать **News Analyst** первым.
2.  **Работа News Analyst:** News Analyst находит последние новости и публикует свой отчет на доске.
3.  **Переоценка Контроллера:** Управление возвращается к Контроллеру. Он читает отчет News Analyst, понимает настроение (sentiment) и следует логике пользователя. Он интеллектуально решает вызвать соответствующего следующего аналитика (**Technical** или **Financial**), полностью пропуская другого.
4.  **Работа Специалиста:** Выбранный аналитик выполняет свою задачу и добавляет свой отчет на доску.
5.  **Финиш Контроллера:** Контроллер видит, что весь необходимый анализ выполнен, и вызывает **Report Writer** для синтеза финального ответа.
6.  **Финальный вызов:** После того как писатель публикует финальный отчет, контроллер видит это и решает сделать **FINISH**.

Этот динамический, оппортунистический рабочий процесс является отличительной чертой правильно функционирующей Blackboard system. Она идеально следовала сложной условной логике пользователя, экономя время и ресурсы.

## Фаза 4: Количественная оценка

Чтобы формализовать сравнение, мы используем LLM-как-Судью для оценки обоих систем по следованию инструкциям и эффективности процесса.

In [ ]:
class ProcessLogicEvaluation(BaseModel):
    """Схема для оценки логического процесса агента."""
    instruction_following_score: int = Field(description="Оценка 1-10, насколько хорошо агент следовал специфическим условным инструкциям пользователя (например, логика 'или/или').")
    process_efficiency_score: int = Field(description="Оценка 1-10, выбрал ли агент наиболее прямой путь и избежал ли ненужной работы.")
    justification: str = Field(description="Краткое обоснование оценок, ссылающееся на конкретные шаги, предпринятые агентом.")

# Используем сильную модель для судейства
judge_llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0).with_structured_output(ProcessLogicEvaluation)

def evaluate_agent_logic(query: str, final_state: dict):
    # Реконструкция упрощенной трассировки для судьи
    trace = ""
    agent_type = "Unknown"
    if 'blackboard' in final_state: # Blackboard агент
        agent_type = "Blackboard"
        trace = "\n---\n".join(final_state['blackboard'])
    else: # Последовательный агент
        agent_type = "Sequential"
        trace = f"1. News Report Generated: {final_state.get('news_report')}\n---\n2. Technical Report Generated: {final_state.get('technical_report')}\n---\n3. Financial Report Generated: {final_state.get('financial_report')}"

    prompt = f"""You are an expert judge of AI agent processes. Your task is to evaluate an agent's performance based on its generated content trace.

**User's Original Task:**
"{query}"

**Agent's Type:** {agent_type}
**Agent's Generated Content Trace:**
```
{trace}
```

**Evaluation Criteria:**
1.  **Instruction Following:** Did the agent respect the conditional logic in the user's task? (e.g., "either a technical analysis... or a financial analysis"). A high score means it followed the logic perfectly. A low score means it ignored it.
2.  **Process Efficiency:** Did the agent avoid doing unnecessary work? A high score means it only ran the required specialists. A low score means it ran specialists that the user's logic explicitly said to skip.

Based on the trace, provide your evaluation.
"""
    return judge_llm.invoke(prompt)

console.print("--- [bold]Оценка процесса Последовательного Агента[/bold] ---")
seq_agent_evaluation = evaluate_agent_logic(dynamic_query, final_seq_output)
console.print(seq_agent_evaluation.dict())

console.print("\n--- [bold]Оценка процесса Системы Blackboard[/bold] ---")
bb_agent_evaluation = evaluate_agent_logic(dynamic_query, final_bb_output)
console.print(bb_agent_evaluation.dict())

--- Evaluating Sequential Agent's Process ---


{'instruction_following_score': 2, 'process_efficiency_score': 3, 'justification': "The agent failed to follow the core conditional instruction ('either/or'). Instead of choosing one path based on the news sentiment, it executed both the technical and financial analyses. This shows a lack of adherence to the user's logic and resulted in an inefficient process by performing unnecessary work."}



--- Evaluating Blackboard System's Process ---


{'instruction_following_score': 10, 'process_efficiency_score': 10, 'justification': 'The agent perfectly followed the user\'s conditional instructions. After the News Analyst reported positive sentiment, the system correctly chose to run the Technical Analyst and completely skipped the Financial Analyst. This demonstrates both flawless instruction following and optimal process efficiency, as no unnecessary work was performed.'}


**Обсуждение вывода оценки:**
Оценки судьи выносят четкий количественный вердикт:

- **Последовательный Агент** получит очень низкую оценку `instruction_following_score` (например, 2/10), потому что он вопиюще проигнорировал условие "или/или". Его `process_efficiency_score` также будет низким (например, 3/10), потому что он выполнил целый анализ, который явно не требовался.
- **Система Blackboard** получит почти идеальные оценки (например, 10/10) и за то, и за другое. Судья признает, что динамическое принятие решений Контроллера позволило системе точно следовать инструкциям пользователя и работать с максимальной эффективностью, активируя только необходимых специалистов.

Эта оценка дает окончательное доказательство того, что для сложных, эмерджентных проблем, где путь вперед зависит от промежуточных результатов, гибкость архитектуры Blackboard значительно превосходит жесткий, предопределенный рабочий процесс.

## Заключение

В этом блокноте мы реализовали и исправили **Blackboard System**, продемонстрировав её значительные преимущества перед последовательной мульти-агентной архитектурой. Внедрив общую память (классную доску) и интеллектуального, учитывающего состояние **Контроллера**, мы создали систему, которая не просто коллаборативна, но также адаптивна и оппортунистична.

Прямое сравнение показало, что для задач с условной логикой способность системы Blackboard выбирать правильного эксперта в нужное время ведет к более эффективному и логически обоснованному процессу. Хотя она требует более сложного Контроллера, эта архитектура является мощным инструментом для решения плохо структурированных проблем реального мира, которые жесткие, линейные рабочие процессы не могут решить эффективно.